In [1]:
#구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import StratifiedGroupKFold

## 케글 데이터로 머신러닝 모델 학습

https://www.kaggle.com/competitions/talkingdata-adtracking-fraud-detection/overview

In [3]:
train_path='/content/drive/MyDrive/ML/fraud_train.csv'
test_path='/content/drive/MyDrive/ML/fraud_test.csv'

In [5]:
df=pd.read_csv(train_path)
df.head()
#is_attributed 컬럼이 타겟 데이터이다.

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0


## 1. 데이터의 확인
- info()
- describe()
- shapoe
- columns

In [11]:
df.info()
# object 데이터(글자와 같은 자연어)를 숫자형태로 변형시켜줘야 활용하기 쉬움. 그리고 이런 변형을 '인코딩'이라 한다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   ip               100000 non-null  int64 
 1   app              100000 non-null  int64 
 2   device           100000 non-null  int64 
 3   os               100000 non-null  int64 
 4   channel          100000 non-null  int64 
 5   click_time       100000 non-null  object
 6   attributed_time  227 non-null     object
 7   is_attributed    100000 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 6.1+ MB


In [10]:
df.describe() # 근데 device와 같은 것의 데이터값은 원래 iphione15 이런 값인데, 이 데이터는 이미 모두 인코딩이 완료된 상태네...

,ip,app,device,os,channel,is_attributed
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000
mean,91255.879670,12.04788,21.771250,22.818280,268.832460,0.002270
std,69835.553661,14.94150,259.667767,55.943136,129.724248,0.047591
min,9.000000,1.00000,0.000000,0.000000,3.000000,0.000000
25%,40552.000000,3.00000,1.000000,13.000000,145.000000,0.000000
50%,79827.000000,12.00000,1.000000,18.000000,258.000000,0.000000
75%,118252.000000,15.00000,1.000000,19.000000,379.000000,0.000000
max,364757.000000,551.00000,3867.000000,866.000000,498.000000,1.000000


In [13]:
df.shape

(100000, 8)

In [18]:
df.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time',
       'is_attributed'],
      dtype='object')

In [24]:
# 각 열마다 가진 데이터에서 유니크한 값의 개수를 셈 -> 열끼리의 불균형 파악 가능

for i in df.columns:
  print(f'{i}의 유니크한 값의 개수: {df[i].nunique()}개\n')

ip의 유니크한 값의 개수: 34857개

app의 유니크한 값의 개수: 161개

device의 유니크한 값의 개수: 100개

os의 유니크한 값의 개수: 130개

channel의 유니크한 값의 개수: 161개

click_time의 유니크한 값의 개수: 80350개

attributed_time의 유니크한 값의 개수: 227개

is_attributed의 유니크한 값의 개수: 2개



In [38]:
# 훈련데이터에서 is_attributed의 값이 고르게 분포하고 있는가?를 확인해봐야 한다.
# 다운로드 했는가? 와 다운로드 안 했는가?의 분포가 5:5 정도로 잘 분포되어 있어야 학습의 의미가 있다.
# 만약 9:1의 비율이라면, 모델이 학습을 안 하고 그냥 다운로드했다고 우겨도 할 말이 없잖아.


df['is_attributed'].value_counts()



# 만약 이렇게 불균형이 심하다면... 골치 아픈 거지.

,count
is_attributed,
0,99773
1,227


In [50]:
x= df[['ip', 'app', 'device', 'os', 'channel']] # 컬럼 중에서 일부만으로 학습을 해보자
y= df['is_attributed']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)


In [40]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
import xgboost as xgb
from xgboost import XGBClassifier

In [51]:
# 그래디언트 부스팅 알고리즘 세팅(옵션, 약한 분류기 몇 개를 사용할 것인가 =10개)
model=GradientBoostingClassifier(n_estimators=10)
model.fit(x_train, y_train)

y_pred=model.predict(x_test)

# 평가지표
- precision_score 정밀도: 양성이라고 한 것 중에 실제로 양성일 확률 (스팸 판단)
- recall_score 재현율: 실제로 양성인 것 중에 양성이라고 할 확률 (암 진단)
- f1_score: 정밀도와 재현율을 모두 사용 2x((p x r)/(p + r))

In [43]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [52]:
precision=precision_score(y_test, y_pred)
print(f'정밀도값: {precision}\n')

recall=recall_score(y_test, y_pred)
print(f'재현율값: {recall}\n')

f1=f1_score(y_test, y_pred)
print(f'f1_score: {f1}\n')

정밀도값: 0.5

재현율값: 0.07142857142857142

f1_score: 0.125



## 3. 성능 개선방법: 훈련을 다르게 시켜보기

- 데이터를 더 많이 모은다.
- feature engineering (원래 데이터를 가공해서 피처를 늘린다던지 nan값을 줄인다던지..)
- 하이퍼 파라미터 튜닝
1. 파라미터: 모델이 학습해서 쌓은 경험치라서 사람이 건들 수 없다.
2. 하이퍼 파라미터: 사람이 모델의 옵션으로써 지정할 수 있는 조건들

- 하이퍼 파라미터 튜닝은 그리드와 랜덤 두 가지 종류로 나눠진다.
1. 그리드 서치: 내가 선정한 옵션 값들로 그리드 (매트릭스)를 만들고 그 조합으로 테스트들을 다 해보고 최선의 조합을 찾기
2. 랜덤 서치: 내가 선정한 범위 안에서 램덤 옵션 값들로 테스트해보고 최선의 조합을 찾기

In [54]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

# 랜덤 서치
from sklearn.model_selection import RandomizedSearchCV

In [66]:
# 그리드 서치 고고씽
# 우선 "내가 선정한 옵션들" <- 내가 어떤 옵션들을 선정할 수 있는지부터 시작
# 사이킷런 사이트의 API에서 GridSearchCV를 검색해보면서 옵션들을 확인한다.
# 아래는 찾은 옵션들.
# estimator = 어떤 모델을 학습시킬 것인지?
# param_grid = 어떤 조합을 해보면 될지? (위에서 미리 정해준 딕셔너리)
# scoring = 평가기준 -> 이 기준을 기반으로 가장 성능이 높은 파라미터 조합을 찾는다


# 하.파 중 실험하고 싶은 것을 임의로 선택하여 몇 개의 값을 미리 넣어줘야 함.
# 'n_estimators'
# 'learning_rate'
param_grid ={
    'n_estimators': [3, 9, 27],
    'learning_rate': [0.1, 0.01, 1]
}

grid_search=GridSearchCV(estimator= GradientBoostingClassifier(),
                         param_grid=param_grid,
                         cv=5,
                         scoring='f1')

In [67]:
grid_search.fit(x_train, y_train)
print(f'Best params: {grid_search. best_params_}')

Best params: {'learning_rate': 0.1, 'n_estimators': 27}


In [80]:
new_model=GradientBoostingClassifier(learning_rate=0.1, n_estimators=27)
new_model.fit(x_train, y_train)

pred=new_model.predict(x_test)

new_precision=precision_score(y_test, pred)
print(f'new 정밀도값: {new_precision}\n')

new_recall=recall_score(y_test, pred)
print(f'new 재현율값: {new_recall}\n')

new_f1=f1_score(y_test, pred)
print(f'new f1_score: {new_f1}\n')


new 정밀도값: 0.2631578947368421

new 재현율값: 0.11904761904761904

new f1_score: 0.16393442622950818



In [76]:
# 랜덤 서치

import random

param_random = {
    'n_estimators': [random.randint(1, 50) for x in range(5)],  # 1에서 50 사이의 정수 생성
    'learning_rate': [random.uniform(0.01, 1) for x in range(5)]  # 0.01에서 1 사이의 실수 생성
}

param_random

{'n_estimators': [21, 22, 20, 4, 46],
 'learning_rate': [0.5855185614798708,
  0.18199504791038604,
  0.1289993794181831,
  0.6137146624054937,
  0.927735354185178]}

In [77]:
random_search=RandomizedSearchCV(estimator=GradientBoostingClassifier(),
                         param_distributions=param_random,
                                 n_iter=5)

random_search.fit(x_train, y_train)
print(f'Best params: {random_search. best_params_}')

Best params: {'n_estimators': 4, 'learning_rate': 0.6137146624054937}


## 머신러닝의 흐름 복습
1. 데이터 수집
2. 데이터 확인 (시각화 등을 동반하여 EDA)
3. 피처 엔지니어링 (피처를 가공하거나 피처를 생성하여 더 나은 자료를 만들거나 피처가 머신러닝에 들어갈 수 있는 형태로 변형하기)
4. 머신러닝 모델을 설택하여 훈련
  - 그리드 서치, 랜덤 서치를 활용
5. 모델을 평가
6. 제출하는데에 써야하는 테스트 데이터 (0.2로 나눈 테스트 데이터말고 따로 제공받은 테스트 데이터)를 3을 다시 적용하고 4의 모델에 넣어서 예측
7. 예측한 데이터를 제출

In [92]:
# 6단계에서 3을 다시 적용하는 것은 생략하고, 4의 모델에서 예측하는 걸 해보려함.
testdf=pd.read_csv(test_path)
testdf.head()

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


In [79]:
testdf.columns

Index(['click_id', 'ip', 'app', 'device', 'os', 'channel', 'click_time'], dtype='object')

In [83]:
testx=testdf[['ip', 'app', 'device', 'os', 'channel']]
new_pred=new_model.predict(testx)

In [84]:
new_pred

array([0, 0, 0, ..., 0, 0, 0])

In [89]:
submission = pd.concat((testdf['click_id'], pd.DataFrame(new_pred)), axis=1)
submission.columns=['click_id', 'is_attributed']
submission

,click_id,is_attributed
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
18790464,18790464,0
18790465,18790465,0
18790466,18790467,0
18790467,18790466,0


In [91]:
submission.to_csv('/content/drive/MyDrive/ML/fraud_submission.csv', index=False)

## 머신러닝의 흐름 복습
1. 데이터 수집
2. 데이터 확인 (시각화 등을 동반하여 EDA)
3. 피처 엔지니어링 (피처를 가공하거나 피처를 생성하여 더 나은 자료를 만들거나 피처가 머신러닝에 들어갈 수 있는 형태로 변형하기)
4. 머신러닝 모델을 설택하여 훈련
  - 그리드 서치, 랜덤 서치를 활용
5. 모델을 평가
6. 제출하는데에 써야하는 테스트 데이터 (0.2로 나눈 테스트 데이터말고 따로 제공받은 테스트 데이터)를 3을 다시 적용하고 4의 모델에 넣어서 예측
7. 예측한 데이터를 제출


여기서 3단계가 중요하다.
요즘은 모델의 옵션을 어떤 걸 선택하느냐보다는 훈련 데이터 자체의 피처 엔지니어링이 중요하다.

In [93]:
df.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0


In [122]:
# 연도, 월일, 시간으로 다 따로 분리해내는 전처리를 진행해보자.

train=df.copy()
train['datetime']=pd.to_datetime(df['click_time']) # 판다스의 투데이트타임 함수를 이용해서 데이터의 클릭타임 열을 시간에 대한 정보로 변환


train['dayofyear']=train['datetime'].dt.dayofyear# 이 날짜가 이 해의 몇 번째 날인가?
train['month']=train['datetime'].dt.month
train['day']=train['datetime'].dt.dayofweek # 월화수목금토일-> 변경
train['hour']=train['datetime'].dt.hour

train.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,datetime,dayofyear,month,day,hour
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0,2017-11-07 09:30:38,311,11,1,9
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0,2017-11-07 13:40:27,311,11,1,13
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0,2017-11-07 18:05:24,311,11,1,18
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0,2017-11-07 04:58:08,311,11,1,4
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0,2017-11-09 09:00:09,313,11,3,9


In [117]:
train['attributed_time'].isnull().sum()
# 흠... 이 컬럼은 nan값이 너무 많다...

99773

In [118]:
# 미니 프로젝트: 앱별로 타켓컬럼 0의 개수 확인하기
app_count=train.groupby('app').size() # app컬럼의 값에 대한 도수분포표
app_count

,0
app,
1,3135
2,11737
3,18279
4,58
5,188
...,...
486,1
536,1
538,1


In [119]:
per_app=train.groupby('is_attributed').size()
per_app

,0
is_attributed,
0,99773
1,227


In [120]:
train.groupby('is_attributed')['app'].value_counts(normalize=True)

is_attributed  app
0              3      0.183166
               12     0.132270
               2      0.117637
               9      0.090044
               15     0.086126
                        ...   
1              48     0.004405
               32     0.004405
               20     0.004405
               60     0.004405
               58     0.004405
Name: proportion, Length: 192, dtype: float64

In [128]:
# 데이터 프레임에서 특정 열을 지우고 학습했을 때 결과가 달라지나도 확인해보자.

train2=train.copy().drop(['click_time',	'attributed_time',	'is_attributed',	'datetime'], axis=1, inplace=None)

In [129]:
train2.head()

,ip,app,device,os,channel,dayofyear,month,day,hour
0,87540,12,1,13,497,311,11,1,9
1,105560,25,1,17,259,311,11,1,13
2,101424,12,1,19,212,311,11,1,18
3,94584,13,1,13,477,311,11,1,4
4,68413,12,1,1,178,313,11,3,9


In [131]:
x_train, x_test, y_train, y_test = train_test_split(train2, df['is_attributed'], test_size=0.2)
new_model = GradientBoostingClassifier(learning_rate=0.1, n_estimators=27)
new_model.fit(x_train, y_train)

new_pred = new_model.predict(x_test)

# Precision, Recall, F1 Score 계산
precision = precision_score(y_test, new_pred)
recall = recall_score(y_test, new_pred)
f1 = f1_score(y_test, new_pred)

# 결과 출력
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Precision: 0.67
Recall: 0.17
F1 Score: 0.27


정밀도가 유의미하게 증가했다.